##  MaskAngleConstraint tester
Note: uses astropy 2.0 and astroplan 0.4.  Numpy is used when possible.

In [1]:
#!/usr/bin/env python3
"""
MaskAngleConstraint tester.
"""

'\nMaskAngleConstraint tester.\n'

In [2]:
import pkg_resources
pkg_resources.require("astropy>=2.0")
pkg_resources.require("astroplan>=0.3")

[astroplan 0.4 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 pytz 2017.2 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 astropy 2.0.2 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 numpy 1.13.3 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 numpy 1.13.3 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 pytest 3.2.1 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 setuptools 36.5.0.post20170921 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 py 1.4.34 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages)]

In [3]:
# from astroplan import download_IERS_A
# download_IERS_A()

In [4]:
from astroplan import Observer, FixedTarget
from astropy.time import Time, TimeDelta

In [5]:
from astroplan import Constraint
from astroplan.constraints import _get_altaz
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import datetime

In [6]:
import sqlite3
from sqlite3 import Error

"""
CREATE TABLE `scores` (
	`key`	TEXT,
	`value`	REAL,
	PRIMARY KEY(`key`)
)
"""
sqlite_file = '/Users/jdgibson/git/QueueScheduler2.0/scores.sqlite' 
table = "scores"
 


In [7]:
# create a database connection
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())
conn.close()

[('scores',)]


In [8]:
def get_score(conn, key):
    """

    """
    c = conn.cursor()
    c.execute("SELECT * FROM scores WHERE key='{}'".format(key)) 
    row = c.fetchone()
    return row

In [9]:
def set_score(conn, key, value):
    """

    """
    c = conn.cursor()
    
    try:
        sql = "REPLACE INTO scores VALUES ('{}', {})".format(key, value)
        if True:
            print("sql: ", sql)
        
        # Can do this with new versions of sqlite.  It does commits automatically.
        with conn:
            conn.execute(sql)

        # Older version.
        # c.execute(sql)
        # Save (commit) the changes
        # conn.commit()
        # conn.close()
    except sqlite3.IntegrityError:
        print('ERROR: ID already exists in PRIMARY KEY column {}'.format("key"))


In [10]:
def get_key(constraint, target, time):
    """
    
    """
    time.format = 'isot'
    key = "{}.{}.{}".format(target, 
                            str(time), 
                            constraint)
    if True:
        print("key:", key)
    return key

In [11]:
def dict_factory(cursor, row):
    d = {}
    for idx,col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

In [12]:
mmto = Observer(longitude=249.11499999999998*u.deg,
                                 latitude=31.688333333333333*u.deg, 
                                 elevation=2608*u.m,
                                 name="mmto",
                                 timezone="America/Phoenix")
times = Time(["2015-08-01 06:00", "2015-08-01 12:00"])

In [13]:
print(mmto)

<Observer: name='mmto',
    location (lon, lat, el)=(-110.88500000000002 deg, 31.688333333333325 deg, 2607.999999999073 m),
    timezone=<DstTzInfo 'America/Phoenix' LMT-1 day, 16:32:00 STD>>


In [14]:
# Read in the table of targets
from astropy.io import ascii
target_table = ascii.read('targets.txt')

In [15]:
# Create astroplan.FixedTarget objects for each one in the table
targets = [FixedTarget(coord=SkyCoord(ra=ra*u.deg, dec=dec*u.deg), name=name)
           for name, ra, dec in target_table]

In [16]:
print(targets)

[<FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>, <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>, <FixedTarget "Albireo" at SkyCoord (ICRS): (ra, dec) in deg ( 292.68033548,  27.95968007)>, <FixedTarget "Algol" at SkyCoord (ICRS): (ra, dec) in deg ( 47.04221855,  40.95564667)>, <FixedTarget "Rigel" at SkyCoord (ICRS): (ra, dec) in deg ( 78.63446707, -8.20163837)>, <FixedTarget "Regulus" at SkyCoord (ICRS): (ra, dec) in deg ( 152.09296244,  11.96720878)>]


In [17]:
k = get_key("MeridianConstraint", targets[0].name, times[0])
if True:
    print("k = ", k)

key: Polaris.2015-08-01T06:00:00.000.MeridianConstraint
k =  Polaris.2015-08-01T06:00:00.000.MeridianConstraint


In [18]:
conn = sqlite3.connect(sqlite_file)
conn.row_factory = dict_factory
value = 0.3424
# conn = sqlite3.connect(sqlite_file, timeout=10)
set_score(conn, k, value)

sql:  REPLACE INTO scores VALUES ('Polaris.2015-08-01T06:00:00.000.MeridianConstraint', 0.3424)


In [19]:
# conn = sqlite3.connect(sqlite_file, timeout=10)
s = get_score(conn, k)
print("score = ", repr(s))

score =  {'key': 'Polaris.2015-08-01T06:00:00.000.MeridianConstraint', 'value': 0.3424}


In [20]:
-

In [21]:
m = MeridianConstraint( mode="sunset",
                 min_alt_degrees=20 * u.deg,
                 max_solar_altitude=-12 * u.deg,
                 grid_times_targets=False, 
                 debug=True)

In [22]:
print("Constraint name: ", repr(type(m).__name__))

Constraint name:  'MeridianConstraint'


In [23]:
m.compute_constraint(times, mmto, targets)

target:  <FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>
key: Polaris.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.3424


sql:  REPLACE INTO scores VALUES ('Polaris.2015-08-01T06:00:00.000.MeridianConstraint', 0.36720391455623835)
key: Polaris.2015-08-01T12:00:00.000.MeridianConstraint
Using saved score. 0.8672034377190802
sql:  REPLACE INTO scores VALUES ('Polaris.2015-08-01T12:00:00.000.MeridianConstraint', 0.8672034377190802)
target:  <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>
key: Vega.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.9485614448785782
sql:  REPLACE INTO scores VALUES ('Vega.2015-08-01T06:00:00.000.MeridianConstraint', 0.9485614448785782)
key: Vega.2015-08-01T12:00:00.000.MeridianConstraint
Using saved score. 0.4486174117801366
sql:  REPLACE INTO scores VALUES ('Vega.2015-08-01T12:00:00.000.MeridianConstraint', 0.4486174117801366)
target:  <FixedTarget "Albireo" at SkyCoord (ICRS): (ra, dec) in deg ( 292.68033548,  27.95968007)>
key: Albireo.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.9765930855605337
sql:  

array([[ 0.36720391,  0.86720344],
       [ 0.94856144,  0.44861741],
       [ 0.97659309,  0.52320695],
       [ 0.34264003,  0.84261654],
       [ 0.16800772,  0.66800573],
       [ 0.24455534,  0.26091625]])

In [24]:
print("targets: ", repr(targets))

targets:  [<FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>, <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>, <FixedTarget "Albireo" at SkyCoord (ICRS): (ra, dec) in deg ( 292.68033548,  27.95968007)>, <FixedTarget "Algol" at SkyCoord (ICRS): (ra, dec) in deg ( 47.04221855,  40.95564667)>, <FixedTarget "Rigel" at SkyCoord (ICRS): (ra, dec) in deg ( 78.63446707, -8.20163837)>, <FixedTarget "Regulus" at SkyCoord (ICRS): (ra, dec) in deg ( 152.09296244,  11.96720878)>]


In [25]:
m = MeridianConstraint( mode="sunset",
                 min_alt_degrees=20 * u.deg,
                 max_solar_altitude=-12 * u.deg,
                 grid_times_targets=True, 
                 debug=True)
start = datetime.datetime.now()
for count in range(10):
    m.compute_constraint(times, mmto, targets)
end = datetime.datetime.now()
print("Elapsed time: ", str(end - start))

target:  <FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>
key: Polaris.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.36720391455623835


sql:  REPLACE INTO scores VALUES ('Polaris.2015-08-01T06:00:00.000.MeridianConstraint', 0.36720391455623835)
key: Polaris.2015-08-01T12:00:00.000.MeridianConstraint
Using saved score. 0.8672034377190802
sql:  REPLACE INTO scores VALUES ('Polaris.2015-08-01T12:00:00.000.MeridianConstraint', 0.8672034377190802)
target:  <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>
key: Vega.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.9485614448785782
sql:  REPLACE INTO scores VALUES ('Vega.2015-08-01T06:00:00.000.MeridianConstraint', 0.9485614448785782)
key: Vega.2015-08-01T12:00:00.000.MeridianConstraint
Using saved score. 0.4486174117801366
sql:  REPLACE INTO scores VALUES ('Vega.2015-08-01T12:00:00.000.MeridianConstraint', 0.4486174117801366)
target:  <FixedTarget "Albireo" at SkyCoord (ICRS): (ra, dec) in deg ( 292.68033548,  27.95968007)>
key: Albireo.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.9765930855605337
sql:  

sql:  REPLACE INTO scores VALUES ('Algol.2015-08-01T06:00:00.000.MeridianConstraint', 0.3426400301964194)
key: Algol.2015-08-01T12:00:00.000.MeridianConstraint
Using saved score. 0.842616540375683
sql:  REPLACE INTO scores VALUES ('Algol.2015-08-01T12:00:00.000.MeridianConstraint', 0.842616540375683)
target:  <FixedTarget "Rigel" at SkyCoord (ICRS): (ra, dec) in deg ( 78.63446707, -8.20163837)>
key: Rigel.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.16800771560381955
sql:  REPLACE INTO scores VALUES ('Rigel.2015-08-01T06:00:00.000.MeridianConstraint', 0.16800771560381955)
key: Rigel.2015-08-01T12:00:00.000.MeridianConstraint
Using saved score. 0.668005728158686
sql:  REPLACE INTO scores VALUES ('Rigel.2015-08-01T12:00:00.000.MeridianConstraint', 0.668005728158686)
target:  <FixedTarget "Regulus" at SkyCoord (ICRS): (ra, dec) in deg ( 152.09296244,  11.96720878)>
key: Regulus.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.24455533642459804
sql:  REPLA

sql:  REPLACE INTO scores VALUES ('Polaris.2015-08-01T06:00:00.000.MeridianConstraint', 0.36720391455623835)
key: Polaris.2015-08-01T12:00:00.000.MeridianConstraint
Using saved score. 0.8672034377190802
sql:  REPLACE INTO scores VALUES ('Polaris.2015-08-01T12:00:00.000.MeridianConstraint', 0.8672034377190802)
target:  <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>
key: Vega.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.9485614448785782
sql:  REPLACE INTO scores VALUES ('Vega.2015-08-01T06:00:00.000.MeridianConstraint', 0.9485614448785782)
key: Vega.2015-08-01T12:00:00.000.MeridianConstraint
Using saved score. 0.4486174117801366
sql:  REPLACE INTO scores VALUES ('Vega.2015-08-01T12:00:00.000.MeridianConstraint', 0.4486174117801366)
target:  <FixedTarget "Albireo" at SkyCoord (ICRS): (ra, dec) in deg ( 292.68033548,  27.95968007)>
key: Albireo.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.9765930855605337
sql:  

sql:  REPLACE INTO scores VALUES ('Algol.2015-08-01T06:00:00.000.MeridianConstraint', 0.3426400301964194)
key: Algol.2015-08-01T12:00:00.000.MeridianConstraint
Using saved score. 0.842616540375683
sql:  REPLACE INTO scores VALUES ('Algol.2015-08-01T12:00:00.000.MeridianConstraint', 0.842616540375683)
target:  <FixedTarget "Rigel" at SkyCoord (ICRS): (ra, dec) in deg ( 78.63446707, -8.20163837)>
key: Rigel.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.16800771560381955
sql:  REPLACE INTO scores VALUES ('Rigel.2015-08-01T06:00:00.000.MeridianConstraint', 0.16800771560381955)
key: Rigel.2015-08-01T12:00:00.000.MeridianConstraint
Using saved score. 0.668005728158686
sql:  REPLACE INTO scores VALUES ('Rigel.2015-08-01T12:00:00.000.MeridianConstraint', 0.668005728158686)
target:  <FixedTarget "Regulus" at SkyCoord (ICRS): (ra, dec) in deg ( 152.09296244,  11.96720878)>
key: Regulus.2015-08-01T06:00:00.000.MeridianConstraint
Using saved score. 0.24455533642459804
sql:  REPLA